# Operator Upgrade Tests

## Setup Seldon Core

Follow the instructions to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
import json
import time

## Install Stable Version

In [ ]:
!kubectl create namespace seldon-system

In [ ]:
!helm upgrade seldon seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --namespace seldon-system --set istio.enabled=true --wait

## Launch a Range of Models

In [ ]:
%%writefile resources/model.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.9.1
          name: classifier
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: REST
    name: example
    replicas: 1

In [ ]:
!kubectl create -f resources/model.yaml

In [ ]:
%%writefile ../servers/tfserving/samples/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      name:  halfplustwo
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

In [ ]:
!kubectl create -f ../servers/tfserving/samples/halfplustwo_rest.yaml

In [ ]:
%%writefile ../examples/models/payload_logging/model_logger.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: model-logs
spec:
  name: model-logs
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier_rest:1.3
          name: classifier
          imagePullPolicy: Always
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: REST
      logger:
        url: http://logger.seldon/
        mode: all
    name: logging
    replicas: 1

In [ ]:
!kubectl create -f ../examples/models/payload_logging/model_logger.yaml

Wait for all models to be available

In [ ]:
def waitStatus(desired):
    for i in range(360):
        allAvailable = True
        failedGet = False
        state = !kubectl get sdep -o json
        state = json.loads("".join(state))
        for model in state["items"]:
            if "status" in model:
                print("model", model["metadata"]["name"], model["status"]["state"])
                if model["status"]["state"] != "Available":
                    allAvailable = False
                    break
            else:
                failedGet = True
        if allAvailable == desired and not failedGet:
            break
        time.sleep(1)
    return allAvailable


actual = waitStatus(True)
assert actual == True

## Count the number of resources

In [ ]:
def getOwned(raw):
    count = 0
    for res in raw["items"]:
        if (
            "ownerReferences" in res["metadata"]
            and res["metadata"]["ownerReferences"][0]["kind"] == "SeldonDeployment"
        ):
            count += 1
    return count


def getResourceStats():
    # Get number of deployments
    dps = !kubectl get deployment -o json
    dps = json.loads("".join(dps))
    numDps = getOwned(dps)
    print("Number of deployments owned", numDps)

    # Get number of services
    svcs = !kubectl get svc -o json
    svcs = json.loads("".join(svcs))
    numSvcs = getOwned(svcs)
    print("Number of services owned", numSvcs)

    # Get number of virtual services
    vss = !kubectl get vs -o json
    vss = json.loads("".join(vss))
    numVs = getOwned(vss)
    print("Number of virtual services owned", numVs)

    # Get number of hpas
    hpas = !kubectl get hpa -o json
    hpas = json.loads("".join(hpas))
    numHpas = getOwned(hpas)
    print("Number of hpas owned", numHpas)

    return (numDps, numSvcs, numVs, numHpas)


(dp1, svc1, vs1, hpa1) = getResourceStats()

## Upgrade to latest

In [ ]:
!helm upgrade seldon ../helm-charts/seldon-core-operator --namespace seldon-system --set istio.enabled=true --wait

In [ ]:
actual = waitStatus(False)
assert actual == False

In [ ]:
actual = waitStatus(True)
assert actual == True

In [ ]:
# Give time for resources to terminate
for i in range(120):
    (dp2, svc2, vs2, hpa2) = getResourceStats()
    if dp1 == dp2 and svc1 == svc2 and vs1 == vs2 and hpa1 == hpa2:
        break
    time.sleep(1)
assert dp1 == dp2
assert svc1 == svc2
assert vs1 == vs2
assert hpa1 == hpa2

In [ ]:
!kubectl delete sdep --all